In [1]:
import h5py,sys, re, os
import pandas as pd
import numpy as np

def encode_seq(seq):
    # convert a DNA sequence to one-hot encoding matrix
    mat = np.zeros((len(seq), 4))
    for i in range(len(seq)):
        if seq[i] == 'A':
            mat[i,0] = 1.0
        elif seq[i] == 'C':
            mat[i,1] = 1.0
        elif seq[i] == 'G':
            mat[i,2] = 1.0
        elif seq[i] == 'T' or seq[i] == 'U':
            mat[i,3] = 1.0
    return(mat) 


def get_motif_positions(sequence, motif):
    # This function get positions of a motif in a query sequence (IUPAC code compatible)
    
    # Convert IUPAC code to regular expression
    iupac_regex = {
        "A": "A",
        "C": "C",
        "G": "G",
        "T": "T",
        "R": "[AG]",
        "Y": "[CT]",
        "S": "[GC]",
        "W": "[AT]",
        "K": "[GT]",
        "M": "[AC]",
        "B": "[CGT]",
        "D": "[AGT]",
        "H": "[ACT]",
        "V": "[ACG]",
        "N": "[ACGT]"
    }
    # Convert the motif to regular expression
    regex = "".join([iupac_regex.get(base, base) for base in motif.upper()])
    
    # Use regular expression to count motif occurrences in sequence
    matches = re.finditer(regex, sequence, re.IGNORECASE)
    
    # Use regular expression to count motif occurrences in sequence
    #count = len(re.findall(regex, sequence, re.IGNORECASE))
        
    return([match.start() for match in matches])


In [6]:
#info = pd.read_table('/lab/solexa_bartel/coffee/Sequencing/Tail-seq/20200121/Motif_insertion/info_motif_insertion_all.txt')
#info['f'] = pd.Series([info.loc[i,'Folder'] + info.loc[i, 'File'] for i in range(info.shape[0])])

#idf = 'XL_oocyte_3UTR_motif_insertion_motif_insertion_'
#file_folder = '/lab/solexa_bartel/coffee/Sequencing/Paper/2024_PALAI/Motif_insertion/XL/CV_L_2000_CDS/'

idf = 'XL_b5v5_3UTR_motif_insertion_motif_insertion_'
file_folder = '/lab/solexa_bartel/coffee/Sequencing/Paper/2024_PALAI/Motif_insertion/XL_b5v5/CV_b5v5_L_84/'

motifs = pd.read_table('/lab/solexa_bartel/coffee/Sequencing/Paper/2024_PALAI/Motif_insertion/motifs.txt', header = None).to_numpy().ravel()

# output file with combined data
out_folder = file_folder + 'Motif_insertion_results_combined/'
os.makedirs(out_folder, exist_ok=True)

In [7]:
# calculate average ism scores from all models
for i_motif, motif in enumerate(motifs):
    print(f"Processing data for motif: {motif}")
    # file list for all 10 models 
    file_lst = [file_folder + 'Model_' + str(j) + '/' + idf + motif + '_prediction_results.hdf5' for j in range(1,11,1)]
    print(f"Total number of input files for motif {motif} is {len(file_lst)}.")
    for i_file, f_in in enumerate(file_lst):
        with h5py.File(f_in, 'r') as f:
            if i_file == 0:
                if i_motif == 0:
                    idx = f['id'][:]
                    seqs = f['seq'][:]
                    if 'utr_bool' in f:
                        utr_bool = f['utr_bool'][:]
                wt_scores = f['wt_scores'][:]
                mi_scores = f['mi_scores'][motif][:]
                mi_diff_scores = f['mi_scores'][motif][:] - f['wt_scores'][:][:,None]
            else:
                wt_scores = wt_scores + f['wt_scores'][:]
                mi_scores = mi_scores + f['mi_scores'][motif][:]
                mi_diff_scores = mi_diff_scores + (f['mi_scores'][motif][:] - f['wt_scores'][:][:,None])
   
    print(f"Calculating the average and save data to a new file...")
    # calculate the average
    avg_wt_scores = wt_scores / len(file_lst)
    avg_mi_scores = mi_scores / len(file_lst)
    avg_mi_diff_scores = mi_diff_scores / len(file_lst)
                     
    # write the average ism to a new file
    with h5py.File(out_folder + motif + '.hdf5', 'w') as f:
        f.create_dataset('id', (idx.shape[0],), dtype = h5py.special_dtype(vlen=str), compression = 'lzf')
        f['id'][:] = idx
        f.create_dataset('seq', (seqs.shape[0],), dtype = h5py.special_dtype(vlen=str), compression = 'lzf')
        f['seq'][:] =  seqs
        f.create_dataset('wt_scores', avg_wt_scores.shape, compression = 'lzf')
        f['wt_scores'][:] = avg_wt_scores
        if 'utr_bool' in globals():
            f.create_dataset('utr_bool', utr_bool.shape, dtype = 'i1', compression = 'lzf')
            f['utr_bool'][:] = utr_bool
        f.create_dataset('mi_scores', avg_mi_scores.shape, compression = 'lzf')
        f['mi_scores'][:] = avg_mi_scores
        f.create_dataset('mi_diff_scores', avg_mi_diff_scores.shape, compression = 'lzf')
        f['mi_diff_scores'][:] = avg_mi_diff_scores


Processing data for motif: AATAAA
Total number of input files for motif AATAAA is 10.
Calculating the average and save data to a new file...
Processing data for motif: ATTAAA
Total number of input files for motif ATTAAA is 10.
Calculating the average and save data to a new file...
Processing data for motif: AGTAAA
Total number of input files for motif AGTAAA is 10.
Calculating the average and save data to a new file...
Processing data for motif: TATAAA
Total number of input files for motif TATAAA is 10.
Calculating the average and save data to a new file...
Processing data for motif: CATAAA
Total number of input files for motif CATAAA is 10.
Calculating the average and save data to a new file...
Processing data for motif: GATAAA
Total number of input files for motif GATAAA is 10.
Calculating the average and save data to a new file...
Processing data for motif: AATATA
Total number of input files for motif AATATA is 10.
Calculating the average and save data to a new file...
Processing da

In [8]:
# calculate the average and standard devdiation of the differential scores for each position of each motif

# This defines the offset to the real 3' end because 'AAA' was appended to the 3' end of each sequence
offset = 3

with open(out_folder + 'Motif_insertion_results_average.txt', 'w') as f:
    f.write('motif\tdis2end\tcount\tavg\tstd\tsem\n')
    for motif in motifs:
        with h5py.File(out_folder + motif + '.hdf5', 'r') as f_in:
            mi_scores = f_in['mi_diff_scores'][:]
            avg_ary = np.nanmean(mi_scores, axis = 0)
            std_ary = np.nanstd(mi_scores, axis = 0)
            count_ary = np.sum(~np.isnan(mi_scores), axis = 0)
            sem_ary = std_ary / (count_ary ** 0.5)
         
            for i in range(avg_ary.shape[0]):
                f.write(motif + '\t' + str(avg_ary.shape[0] - i - 1 - offset) + '\t' + str(count_ary[i]) + '\t' + f'{avg_ary[i]:.3f}' + 
                        '\t' + f'{std_ary[i]:.3f}' + '\t' + f'{sem_ary[i]:.3f}' + '\n')
                    
        
    

In [5]:
# calculate the average and standard devdiation of the differential scores for each position of each motif
# mutually mask top motifs 
# This defines the offset to the real 3' end because 'AAA' was appended to the 3' end of each sequence
offset = 3
sele_motifs = ['AATAAA', 'ATTAAA', 'TTTTA']
with open(out_folder + 'Motif_insertion_results_average_select_motifs_mutural_masked.txt', 'w') as f:
    f.write('motif\tdis2end\tcount\tavg\tstd\tsem\n')
    for motif in sele_motifs:
        with h5py.File(out_folder + motif + '.hdf5', 'r') as f_in:
            mi_scores = f_in['mi_diff_scores'][:]
            seqs = f_in['seq'][:]
            
            # mask overlapping positions
            for i in range(len(seqs)):
                mask_pos_lst = []
                
                for mask_motif in sele_motifs:
                    if mask_motif != motif:
                        pos_lst = get_motif_positions(seqs[i].decode('utf-8'), mask_motif)
                        for pos in pos_lst:   
                            mask_pos_lst.extend(range(max(0, pos - len(motif) - 1), min(pos + len(mask_motif), len(seqs[i]) - len(motif) + 1)))

                mask_pos_ary = np.unique(np.array(mask_pos_lst))
                if len(mask_pos_ary) != 0:
                    mi_scores[i,mask_pos_ary] = np.nan
            
            # calculate statistics
            avg_ary = np.nanmean(mi_scores, axis = 0)
            std_ary = np.nanstd(mi_scores, axis = 0)
            count_ary = np.sum(~np.isnan(mi_scores), axis = 0)
            sem_ary = std_ary / (count_ary ** 0.5)
         
            for i in range(avg_ary.shape[0]):
                f.write(motif + '\t' + str(avg_ary.shape[0] - i - 1 - offset) + '\t' + str(count_ary[i]) + '\t' + f'{avg_ary[i]:.3f}' + 
                        '\t' + f'{std_ary[i]:.3f}' + '\t' + f'{sem_ary[i]:.3f}' + '\n')
                    
        
    

In [9]:
# mask top PAS motifs and re-calculate the insertion outcome

# This defines the offset to the real 3' end because 'AAA' was appended to the 3' end of each sequence
offset = 3

sele_motif_lst = [motif for motif in motifs if len(motif) == 6 and motif not in ['AATAAA']]
with open(out_folder + 'Motif_insertion_top_PAS_masked_results_average.txt', 'w') as f:
    f.write('motif\tdis2end\tcount\tavg\tstd\tsem\n')
    for motif in sele_motif_lst:
        with h5py.File(out_folder + motif + '.hdf5', 'r') as f_in:
            mi_scores = f_in['mi_diff_scores'][:]
            seqs = f_in['seq'][:]

            # mask overlapping positions
            for i in range(len(seqs)):
                if motif == 'ATTAAA':
                    pos_lst = get_motif_positions(seqs[i].decode('utf-8'), 'AATAAA')
                else:
                    pos_lst = get_motif_positions(seqs[i].decode('utf-8'), 'AWTAAA')
                mask_pos_lst = []
                for pos in pos_lst:   
                    mask_pos_lst.extend(range(max(0, pos - len(motif) - 1), min(pos + len(motif), len(seqs[i]) - len(motif) + 1)))
                mask_pos_ary = np.unique(np.array(mask_pos_lst))
                if len(mask_pos_ary) != 0:
                    mi_scores[i,mask_pos_ary] = np.nan


            # calculate statistics
            avg_ary = np.nanmean(mi_scores, axis = 0)
            std_ary = np.nanstd(mi_scores, axis = 0)
            count_ary = np.sum(~np.isnan(mi_scores), axis = 0)
            sem_ary = std_ary / (count_ary ** 0.5)

            for i in range(avg_ary.shape[0]):
                f.write(motif + '\t' + str(avg_ary.shape[0] - i - 1 - offset) + '\t' + str(count_ary[i]) + '\t' + f'{avg_ary[i]:.3f}' + 
                        '\t' + f'{std_ary[i]:.3f}' + '\t' + f'{sem_ary[i]:.3f}' + '\n')
        
        

In [10]:
# for insertions of the CPE motif (UUUUA), examine its relative positoin to the PAS (if exists)

# This defines the offset to the real 3' end because 'AAA' was appended to the 3' end of each sequence
offset = 3

# The PAS motif must be within the distance of the 3' ends
pas2end_cutoff = 100

# the relative positions must be with the distance
relative_dis_cutoff = 100

with open(out_folder + 'Motif_insertion_average_CPE2PAS.txt', 'w') as f, h5py.File(out_folder + 'TTTTA' + '.hdf5') as f_in:
    f.write('pA_id\tdis2end\tdis2pas\ttl_diff\n')
    mi_scores = f_in['mi_diff_scores'][:]
    ids = f_in['id'][:]
    n, l = mi_scores.shape
    for i in range(n):
        pos_lst = get_motif_positions(f_in['seq'][i].decode('utf-8'), 'AWTAAA')
        if len(pos_lst) > 0 and (l - offset - pos_lst[-1]) < pas2end_cutoff:
            pos_lst_filtered = [pos for pos in np.arange(l)[~np.isnan(mi_scores[i,:])] if abs(pos - pos_lst[-1]) <= relative_dis_cutoff]
            for j in pos_lst_filtered:
                f.write(ids[i].decode('utf-8') + '\t' + 
                        str(l - j - 1 - offset) + '\t' + 
                        str(pos_lst[-1] - j) + '\t' + 
                        f"{mi_scores[i,j]:.3f}" + '\n')
